In [ ]:
!pip install pyspark
!pip install langdetect
!pip install pyspark_dist_explore
!pip install tweepy
!pip install textblob

In [ ]:
import os
import pandas as pd
import numpy as np

import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.sql.types import FloatType
from textblob import TextBlob
from langdetect import detect
import re

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def langdetect1(tweet):
    try:
        if detect(tweet)=='en':
            return 1
        else:
            return 0
    except:
        return 0

def clean_tweet1(tweet): 
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
    
def sentiment_score1(tweet):
    return TextBlob(tweet).sentiment.polarity

def get_sentiment1(score): 
        if score > 0: 
            return 4
        elif score == 0: 
            return 2
        else: 
            return 0
        
@udf(returnType=IntegerType())
def langdetect(tweet):
    try:
        if detect(tweet)=='en':
            return 1
        else:
            return 0
    except:
        return 0
@udf(returnType=StringType())
def clean_tweet(tweet): 
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
    
@udf(returnType=FloatType())
def sentiment_score(tweet):
    return TextBlob(tweet).sentiment.polarity

@udf(returnType=IntegerType())
def get_sentiment(score): 
        if score > 0: 
            return 4
        elif score == 0: 
            return 2
        else: 
            return 0

In [9]:
df1=pd.read_csv("../input/bitcoin-tweets-20160101-to-20190329/tweets.csv",sep=";",encoding='utf-8')

df1=df1['text']
df1=pd.DataFrame(df1)
df1=df1.dropna()
df['text'] = df['text'].str.strip()
df1=df1[(df1.text!="")]

df1['englishornot']=df1['text'].map(langdetect1)
df1=df1[(df1.englishornot!=0)]
df1['cleantweet']=df1['text'].map(clean_tweet1)
df1['polarity']=df1['cleantweet'].map(sentiment_score1)
df1['output']=df1['polarity'].map(get_sentiment1)
df1.head()

KeyboardInterrupt: 

In [ ]:
spark = SparkSession.builder.master("local[10]").appName("Twitter-Sentiment-Analysis").getOrCreate()
df=spark.read.csv(path="../input/bitcoin-tweets-20160101-to-20190329/tweets.csv",sep=";",header=False).cache()

df = df.select('text')
df = df.na.drop()
df = df.withColumn('text', trim(col('text')))
df = df.filter(col('text')!="")

df=df.withColumn("englishornot", langdetect(col("_c5")))
df = df.filter(col('englishornot')!=0)
df=df.withColumn("cleantweet", clean_tweet(col("_c5")))
df=df.withColumn("polarity", sentiment_score(col("cleantweet")))
df=df.withColumn("output", get_sentiment(col("polarity")))
df.show()

In [ ]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 

class TwitterClient(object): 
    def __init__(self): 
        ''' 
        Class constructor or initialization method. 
        '''
        # keys and tokens from the Twitter Dev Console 
        consumer_key = 'munfQBcIMsJoV78LDDG4QW62B'
        consumer_secret = 'F4QCy5vOkpewpdsX4nqi9EMfS3GujzfOhq0TY3ZvWtpqb5uTF'
        access_token = '1323622428619075585-HQIJsjuGD5FaUdT1XOhKvbdAB2WVLL'
        access_token_secret = 'CV02QISs8Vb9Gz2m38mFQ7Jpn33pyjM5ZqUVOCt5ss4BS'

        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 

    def clean_tweet(self, tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

    def get_tweet_sentiment(self, tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'

    def get_tweets(self, query, count = 10): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 

        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count) 

            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 

                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 

                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 

            # return parsed tweets 
            return tweets 

        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 

def main(): 
    # creating object of TwitterClient Class 
    api = TwitterClient() 
    # calling function to get tweets 
    tweets = api.get_tweets(query = 'Narendra Modi', count = 200) 

    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets 
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
    # percentage of neutral tweets 
    print("Neutral tweets percentage: {} % \ ".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets))) 

    # printing first 5 positive tweets 
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:10]: 
        print(tweet['text']) 

    # printing first 5 negative tweets 
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:10]: 
        print(tweet['text']) 

if __name__ == "__main__": 
# calling main function 
    main()